# Телемаркетинг

Надо помочь коллегам с продажами продуктов через телемаркетинг. 

## Задача:
Проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе. Вывести id полученных пользователей через запятую.

## Особенности:
1. продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM);
2. если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Загрузим даннеы о продажах и посмотрим на них
df1 = pd.read_csv('C:/Users/pakievskij/tm_sales_1.csv', sep=';')
df2 = pd.read_csv('C:/Users/pakievskij/tm_sales_2.csv', sep=';')
df3 = pd.read_csv('C:/Users/pakievskij/tm_sales_3.csv', sep=';')

In [3]:
df1

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25


In [4]:
df2

,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,3,id5416547,1743,17-03-2020 10:17
1,4,NaN,7421,12-03-2020 11:25
2,7,8362218,9879,05-03-2020 11:42
3,2,2185490,3210,16-03-2020 16:28


In [5]:
df3

,FILIAL_ID,PROD_ID,SUBS_ID,ACT_DTTM
0,3,1499,id5764122,18-03-2020 15:44
1,6,3020,id7642700,15-03-2020 14:21
2,2,5677,id1374509,17-03-2020 11:48


В данных есть две проблемы:
1. Столбцы в файлах идут в случайном порядке.  
2. При заполнении данных иногда не вносился полный id. Иногода это просто цифры без 'id' в начале.

In [6]:
# приведём данные в df2 и df3 к формату стобцов df1
df2 = df2[['SUBS_ID', 'FILIAL_ID', 'PROD_ID', 'ACT_DTTM']]
df3 = df3[['SUBS_ID', 'FILIAL_ID', 'PROD_ID', 'ACT_DTTM']]

In [12]:
full_df = pd.concat([df1, df2, df3])

In [13]:
# проверим данные на пропущенные значения
full_df.isna().sum()

SUBS_ID      1
FILIAL_ID    0
PROD_ID      0
ACT_DTTM     0
dtype: int64

In [15]:
# избавимся от строк с пропущеными значениями:
full_df =full_df.dropna().reset_index(drop=True)

In [16]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   SUBS_ID    9 non-null      object
 1   FILIAL_ID  9 non-null      int64 
 2   PROD_ID    9 non-null      int64 
 3   ACT_DTTM   9 non-null      object
dtypes: int64(2), object(2)
memory usage: 416.0+ bytes


In [17]:
# Проверим, что данный по id филиала и продукта без очевидно неправильных значений. Нули и отрицательные числа.
full_df.describe()

,FILIAL_ID,PROD_ID
count,9.000000,9.000000
mean,3.666667,4080.666667
std,2.000000,2757.213312
min,1.000000,1499.000000
25%,2.000000,1954.000000
50%,3.000000,3210.000000
75%,5.000000,5677.000000
max,7.000000,9879.000000


In [10]:
full_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,8362218,7,9879,05-03-2020 11:42
5,2185490,2,3210,16-03-2020 16:28
6,id5764122,3,1499,18-03-2020 15:44
7,id7642700,6,3020,15-03-2020 14:21
8,id1374509,2,5677,17-03-2020 11:48


In [56]:
# Добавим id там, где его нет в начале строки
subs = []
for i in full_df['SUBS_ID']:
    if str(i).startswith('id'):
        i
    else:
        i = 'id' + str(i)
    subs.append(i)
full_df['SUBS_ID'] = subs


In [57]:
full_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
4,id8362218,7,9879,05-03-2020 11:42
5,id2185490,2,3210,16-03-2020 16:28
6,id5764122,3,1499,18-03-2020 15:44
7,id7642700,6,3020,15-03-2020 14:21
8,id1374509,2,5677,17-03-2020 11:48


Данные по продажам преведены к нужному виду. Теперь можно соеденить их с логами.

In [105]:
df_logs = pd.read_csv('C:/Users/pakievskij/prod_activations_logs.csv', sep=';')

In [106]:
df_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
3,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
2,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
6,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
2,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [107]:
merged = full_df.merge(df_logs, how='inner', on=['SUBS_ID', 'PROD_ID'])

In [108]:
merged

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,20-03-2020 14:59,20-03-2020 14:59,01-12-2020 00:00
1,id7646509,5,6431,19-03-2020 13:00,19-03-2020 13:00,19-03-2020 13:03
2,id5416547,3,1743,17-03-2020 10:17,17-03-2020 10:17,25-03-2020 11:00
3,id2185490,2,3210,16-03-2020 16:28,16-03-2020 16:28,01-12-2020 00:00
4,id7642700,6,3020,15-03-2020 14:21,15-03-2020 14:21,15-03-2020 23:42


Теперь отфильтруем данные по условию действия подключенной услуги не мении 5 минут.

In [60]:
merged.dtypes

SUBS_ID       object
FILIAL_ID      int64
PROD_ID        int64
ACT_DTTM      object
START_DTTM    object
END_DTTM      object
dtype: object

Необходимы нам строки имеют тип 'object'. Для расчёта разницы во времени их необходимо перевсти к типу дата.

In [64]:
# для начала преобразуем столбцы в даты, специально указывая исходный формат даты для распознавания
merged[['START_DTTM','END_DTTM','ACT_DTTM']] = \
    merged[['START_DTTM','END_DTTM','ACT_DTTM']].apply(pd.to_datetime, format='%d-%m-%Y %H:%M')

In [122]:
# отбираем те записи в которых время до отключения услуги соотетствует нашему условию
merged = merged.loc[(merged.END_DTTM - merged.START_DTTM) > pd.Timedelta('5 min')]
merged

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-12-01 00:00:00
2,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00
3,id2185490,2,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-12-01 00:00:00
4,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00


In [121]:
# Получаем список id пользователей в нужном формате
', '.join(merged.sort_values('SUBS_ID').SUBS_ID)

'id2185490, id4651830, id5416547, id7642700'